# Predict Missing Markers Trajectories

**References:**
- Federolf PA (2013) *A Novel Approach to Solve the “Missing Marker Problem” in Marker-Based Motion Analysis That Exploits the Segment Coordination Patterns in Multi-Limb Motion Data.* PLOS ONE 8(10): e78689. <a href="https://doi.org/10.1371/journal.pone.0078689">https://doi.org/10.1371/journal.pone.0078689</a>
- Gløersen Ø, Federolf P (2016) *Predicting Missing Marker Trajectories in Human Motion Data Using Marker Intercorrelations.* PLOS ONE 11(3): e0152616. <a href="https://doi.org/10.1371/journal.pone.0152616">https://doi.org/10.1371/journal.pone.0152616</a>

**Author:** Robbin Romijnders

Marker-based motion analysis generally suffers from loss of marker information, for example due to occlusion or marker detachment. Naive approaches to fillings gaps in the marker trajectories are based on linear or spline interpolation, however these are mainly suitable for gaps of short duration. In **Federolf (2013)** and **Gløersen and Federolf (2016)** a method was proposed that takes advantage of intercorrelations between neighboring markers. The MATLAB source code is available as the Supporting Information online:  
<a href="https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0152616#sec018">https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0152616#sec018</a>.

Here, we have implemented the methods in Python for further use.

## Prerequisites

In [70]:
# Load libraries
from lib.utils import _load_file
from lib.preprocessing import _predict_missing_markers
import os
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [68]:
# Load data
PARENT_FOLDER = "./data/test"  # set parent folder
filename = "WalkL_0001.mat"    # set filename
data_gaps = _load_file(os.path.join(PARENT_FOLDER, filename))     # load original gap-free data
data_full = _load_file(os.path.join(PARENT_FOLDER, "WalkL.mat"))  # load data with gaps
print(f"Shape of full data: {data_full.shape}")
print(f"Shape of gapped data: {data_gaps.shape}")

Shape of full data: (4300, 111)
Shape of gapped data: (4300, 111)


## Processing

In [72]:
# Call method to predict missing marker trajectoroes
data_filled = _predict_missing_markers(data_gaps, method="R1")
print(f"Shape of filled data: {data_filled.shape}")

Shape of filled data: (4300, 111)


In [61]:
# Identify channels (i.e. columns) and time steps (i.e. rows) with missing markers data (represented by NaN)
ix_channels_with_gaps, = np.nonzero(np.any(np.isnan(data_gaps), axis=0))
ix_time_steps_with_gaps, = np.nonzero(np.any(np.isnan(data_gaps), axis=0))
ix_markers_with_gaps = ( ix_channels_with_gaps[2::3] // 3 )

In [74]:
# Plot trajectories for a given marker
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

for ix in ix_markers_with_gaps[7:8]:
    # X coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0]), y=data_filled[:,ix*3], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=1, col=1)

    # Y coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3+1], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3+1], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0]), y=data_filled[:,ix*3+1], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=2, col=1)

    # Z coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3+2], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=3, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3+2], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0]), y=data_filled[:,ix*3+2], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=3, col=1)
fig.update_layout(showlegend=False, width=960, height=540)
fig.show()